<div style="width: 100%; clear: both; font-family: Verdana;">
<div style="float: left; width: 50%;font-family: Verdana;">
<img src="https://github.com/Eligoze/att-rci-internal/blob/qa/RCI_DataAnalysis/eda/doc/att-logo1.png" align="left">
</div>
<div style="float: right; width: 200%;">
<p style="margin: 0; padding-top: 20px; text-align:right;color:rgb(193, 38, 184)"><strong>Axity - AT&T.
    Ciclo de vida de elementos de inventario</strong></p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Exploratory Data Analysis
## Ciclo de vida de elementos de inventario.
### Axity - AT&T.

## Descripción
Analizaremos los datos de las fuentes de inventarios de AT&T con un tratamiento estadístico descriptivo para hacer el tracking del ciclo de vida de los elementos de red. Se creará un EDA enfocado a la salida de almacén. Serán documentados los catálogos propuestos junto a su respectivo tratamiento de datos. La fuente que corresponde a este análisis es:

* **NOMBRE DE SOURCE**

Primero cargamos las librerías necesarias.

#### Conectando al Datalake

In [ ]:
import os
os.environ['JAVA_HOME'] = '/usr/java/jdk1.8.0_162'
os.environ['SPARK_HOME'] = '/opt/cloudera/parcels/CDH-6.2.0-1.cdh6.2.0.p0.967373/lib/spark'
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import HiveContext

In [ ]:
conf = SparkConf().setAppName('Segregacion')  \
    .setMaster('yarn') 
spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext
sqlContext = HiveContext(sc)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandasql
%matplotlib inline

from bokeh.io import show, output_notebook 
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category20_11, Category20c_20
output_notebook()

### Recolección de los datos: 

In [ ]:
#EN CASO DE REQUERIR HACER PARTICIONES SE HACE MANUAL:

#df_01=spark.sql("SELECT * FROM tx_").toPandas()
#df_02=spark.sql("SELECT * FROM tx_").toPandas()
#frames=[df_01,df_02]
#df=pd.concat(frames,ignore_index=True)

In [ ]:
df=spark.sql("SELECT * FROM tx_").toPandas()

Hemos recolectado los campos a analizar de la fuente NOMBRE_SOURCE.

## NOMBRE DE LA PARTE DEL CICLO DE VIDA
Una muestra de la fuente SOURCE

In [ ]:
df.head(5)

### Diccionario de datos.
A continuación se enlistan los campos de la fuente con una breve descripción de negocio.  

* **campo**: Definición del negocio.
* **campo**: Definición del negocio.
* **filedate**: Fecha de carga del archivo.
* **filename**: Nombre del archivo cargado.
* **hash_id**: Identificador único Hash.
* **source_id**: Fuente de archivo.
* **registry_state**: Timestamp de carga.
* **datasetname**: Nombre del ....
* **timestamp**: Fecha de carga.
* **transaction_status**: Estatus de carga.
* **year**: Año del archivo.
* **month**: Mes del archivo.
* **day**: Día del archivo.

### 2. Descripción de las fuentes.
En este apartado se hará una descripción a detalle de las fuentes para una mejor comprensión de los datos. Por cada fuente se mostrarán los tipos de datos, tamaño de la fuente, es decir, su dimensionalidad y una estadística descriptiva, en ese orden.

In [ ]:
campos=df.columns
print('Columnas de la fuente SOURCE son: ',list(campos))
pd.DataFrame(df.dtypes,columns=['Tipo de objeto SOURCE'])

In [ ]:
print('renglones = ',df.shape[0],' columnas = ',df.shape[1])

In [ ]:
#Pasamos las columnas que queremos ver en nuestro describe:
NOrelevantes=['filedate', 'filename', 'hash_id', 'sourceid',
              'registry_state', 'datasetname', 'timestamp',
              'transaction_status', 'year', 'month', 'day']
relevantes=[v for v if v not in NOrelevantes]

df[relevantes].describe(include='all')

#### Haremos anotaciones sobre lo que se puede observar de la tabla describe:

* NOTAS OBSERVABLES DEL DESCRIBE, SEAN POSIBLES CATÁLOGOS,CAMPOS ÚNICOS, NULOS O CAMPOS SUCIOS (CUALQUIER COSA OBSRVABLE DE INTERÉS)
* NOTAS OBSERVABLES DEL DESCRIBE, SEAN POSIBLES CATÁLOGOS,CAMPOS ÚNICOS, NULOS O CAMPOS SUCIOS (CUALQUIER COSA OBSRVABLE DE INTERÉS)
* NOTAS OBSERVABLES DEL DESCRIBE, SEAN POSIBLES CATÁLOGOS,CAMPOS ÚNICOS, NULOS O CAMPOS SUCIOS (CUALQUIER COSA OBSRVABLE DE INTERÉS)
* NOTAS OBSERVABLES DEL DESCRIBE, SEAN POSIBLES CATÁLOGOS,CAMPOS ÚNICOS, NULOS O CAMPOS SUCIOS (CUALQUIER COSA OBSRVABLE DE INTERÉS)

#### Se proponen catálogos derivados de la fuente SOURCE con los siguientes campos:
    
* **CAMPO**: DESCRIPCIÓN NEGOCIO.
* **CAMPO**: DESCRIPCIÓN NEGOCIO.  

Estos catálogos nos ayudarán a mapear todos los diferentes proyectos que existen en los cuales hay un activo.


### 3. Exploración de los datos.
De acuerdo al análisis anterior, procedemos a hacer una exploración de los datos con gráficas, de manera particular nos enfocaremos en los catálogos.

#### Para empezar, se hará una limpieza general a los datos:

In [ ]:
df.replace('null',np.NaN,inplace=True)
df.replace('NA',np.NaN,inplace=True)
#Se puede hacer más y por columna en caso de ser necesario

### Primer catálogo: *CAMPO*

Empezaremos con el catálogo de CAMPO. BREVE DESCRIPCIÓN DE EL PROCESO QUE SE REALIZARÁ. Este catálogo será llamado después en el apartado de catálogos. Nuestra intención por el momento es simplemente explorar los datos.

In [ ]:
#Revisamos frecuencias:
CAMPO=pd.DataFrame(df.CAMPO.value_counts())

#Visualización:
fig=plt.figure()
ax0=fig.add_subplot(1,2,1)
ax1=fig.add_subplot(1,2,2)

#Subplot1: Bar chart
CAMPO.plot(kind='bar',figsize=(10,6),colormap='rainbow_r',ax=ax0)
ax0.set_xlabel(u'NOMBRE CAMPO')
ax0.set_ylabel('Frecuencia')
ax0.set_title(u'NOMBRE DEL CHART')

#Subplot2: Pie chart
#La lista explode debe ser ajustada manualmente contra el número de elementos del catálogo
explode_list=[.2,0,0,0,0,0.1,0.15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] #Cada entrada corresponde a un registro del catálogo

color_list=['royalblue','lightcoral','springgreen','powderblue','oldlace','palegoldenrod',
            'peachpuff','rebeccapurple','tomato','slateblue','silver','mediumaquamarine',
            'mediumblue','mediumorchid','turquoise','powderblue','oldlace','palegoldenrod',
            'peachpuff','rebeccapurple','tomato','slateblue','powderblue']

CAMPO['CAMPO'].plot(kind='pie',         
                    figsize=(15,8),
                    autopct='%1.1f%%',
                    startangle=90,
                    shadow=True,
                    labels=None,
                    pctdistance=1.14,
                    colors=color_list,
                               ax=ax1,
                    explode=explode_list)
ax1.set_title(u'NOMBRE DEL CHART',y=1.12)
ax1.axis('equal')
ax1.legend(labels=areas.index,loc='upper left')

plt.show()

Podemos observar en primer lugar, que se necesitan reglas de limpieza, existen outliers que al parecer son datos sucios.
Se llamará al catálogo limpio en el apartado de catálogos.

#### Visualización de los datos de trazabilidad: 

In [ ]:
pd.DataFrame(df.serie.value_counts()[:15])

In [ ]:
pd.DataFrame(df.activo.value_counts()[:15])

#### Es de interés haber observado los datos que se usaran para la trazabilidad, NOTAS OBSERVABLES. 
Podemos utilizar los siguientes queries como ejemplos para visualizar en HUE:
* Activo: `QUERY PARA CORROBORAR EN HUE` 
* Serie: `QUERY PARA CORROBORAR EN HUE`

<div style="width: 100%; clear: both; font-family: Verdana;">
<h2>4. Calidad de los datos</h2>
    <p> Se documentará la calidad de los datos y analizará las variables que necesitan tratamiento con la ayuda de visualizaciones y tablas.
    </p>
</div>

<div style="width: 100%; clear: both; font-family: Verdana;">
<h4>Missings Values</h4>
    <p>Los missings values son los valores faltantes en el conjunto de datos que se refieren a aquellos campos que están vacíos o no tienen valores asignados, estos generalmente ocurren debido a errores de entrada de datos, fallas que ocurren con los procesos de recopilación de datos y, a menudo, al unir varias columnas de diferentes tablas encontramos una condición que conduce a valores faltantes. Existen numerosas formas de tratar los valores perdidos, los más fáciles son reemplazar el valor perdido con la media, la mediana, la moda o un valor constante (llegamos a un valor basado en el conocimiento del dominio) y otra alternativa es eliminar la entrada desde el conjunto de datos en sí.
    </p>
</div>

<div style="width: 100%; clear: both; font-family: Verdana;">
    <p> Calculamos el porcentaje de NA's que tiene la fuente por columna y el porcentaje de los missings.
        </p>
</div>

In [ ]:
nas=df[relevantes].isna().sum()
porcentaje_nas=nas/df[relevantes].isna().count()

columnas=list(porcentaje_nas.keys())
counts_nas=list(porcentaje_nas.values)

#Para el siguiente comando, en el parámetro "color":
#Dependiendo el número de columnas se escoge un pallete, este debe ser cargado en la sección de librerías,
#Sólo se añade a la parte from bokeh.palettes import Category20c_20  colocando una ","
#http://docs.bokeh.org/en/1.3.2/docs/reference/palettes.html
#Se recomienda no poner más de 20 columnas. 
source=ColumnDataSource(dict(columnas=columnas, counts_nas=counts_nas, color=Category20c_20))

p=figure(x_range=columnas, plot_height=300, plot_width=850, y_range=(0,1), title='Porcentaje de nas por columna')
p.vbar(x='columnas',top='counts_nas',width=.7, color='color', legend='columnas', source=source)

p.xgrid.grid_line_color = None
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

show(p)
pd.DataFrame(porcentaje_wlog_nas*100,columns=['Porcentaje de NAs'])

#### Visualización de datos NOT NULL: 

In [ ]:
notmiss=(1-porcentaje_nas)*100

columnas=list(notmiss.keys())
counts_nas=list(notmiss.values)

#Mismo aplica aquí para color
source=ColumnDataSource(dict(columnas=columnas, counts_nas=counts_nas, color=Category20c_20))

p=figure(x_range=columnas, plot_height=300, plot_width=850, y_range=(0,100), 
         title='Porcentaje de not-nulls por columna')
p.vbar(x='columnas',top='counts_nas',width=.5, color='color', legend='columnas', source=source)

p.xgrid.grid_line_color = None
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

show(p)

pd.DataFrame(notmiss,columns=['Porcentaje de Not nulls'])

NOTAS SOBRE LO OBSERVADO EN MISSINGS

#### 4.2 Data Errors
* **CAMPO**: comentarios sobre errores de los datos(Ejemplos de datos sucios, mal formato).
* **CAMPO**: comentarios sobre errores de los datos(Ejemplos de datos sucios, mal formato).

### 5. Catálogos.

#### Catálogo de PRIMER CATÁLOGO:

In [ ]:
Catalogo_CAMPO=pd.DataFrame(df.CAMPO.unique())
Catalogo_CAMPO.columns=['CAMPO']

#Remover los sucios
#Esta parte depende del estado del catálogo, deben quedar sólamente los valores limpios, un ejemplo puede ser:
#Catalogo_CAMPO.drop(labels=[67,346,369,279,171,313],axis=0,inplace=True)

#Se le da 
Catalogo_CAMPO.reset_index(drop=True)
Catalogo_CAMPO.dropna(inplace=True)
Catalogo_CAMPO.sort_values(by='CAMPO').head(10)

### 6. Preparación de los datos.
Para la preparación de los datos crearemos las reglas de calidad o estándares observados en el apartado anterior de acuerdo a la calidad de datos obtenidos: 

* **CAMPO**: REGLA DE LIMPIEZA.
* **CAMPO**: REGLA DE LIMPIEZA.

### 7. Métricas KPI.
Se mostrarán los KPIs generados. 

In [ ]:
Total_Elementos=df.shape[0]
Total_Elementos

In [ ]:
df.replace(np.NaN,'vacio',inplace=True)

#### Total Elementos Trazables

In [ ]:
Total_Tr=df.loc[(df.serie!='vacio') | (df.activo!='vacio')].shape[0]
Total_Tr

#### Total Elementos NO Trazables

In [ ]:
Total_NOTr=Total_Elementos-Total_Tr
Total_NOTr

#### Total Elementos Trazables Únicos

In [ ]:
Total_Tr_Unic=df[['serie','activo']].loc[(df.serie!='vacio') | (df.activo!='vacio')].drop_duplicates().shape[0]
Total_Tr_Unic

#### Total de elementos trazables duplicados

In [ ]:
Total_Tr_Dupli=Total_Tr-Total_Tr_Unic
Total_Tr_Dupli

#### EN CASO DE EXISTIR NOMBRE DE ALMACENES:

#### Total de elementos en almacén Trazables Únicos en NOMBREALMACEN

In [ ]:
NOMBREALMACEN_Tr_Unic=df[['serie','etiqueta']].loc[(df.almacen=='NOMBREALMACEN')&((df.serie!='vacio') | (df.etiqueta!='vacio'))].drop_duplicates().shape[0]
NOMBREALMACEN_Tr_Unic

#### Total de elementos en almacén Trazables Únicos con NSerie, con Nactivo

In [ ]:
Total_Tr_Unic_CS_CA=df[['serie','etiqueta']].loc[(df.serie!='vacio') & (df.etiqueta!='vacio')].drop_duplicates().shape[0]
Total_Tr_Unic_CS_CA

#### Total de elementos en almacén Trazables Únicos con NSerie, sin Nactivo 

In [ ]:
Total_Tr_Unic_CS_SA=df[['serie','etiqueta']].loc[(df.serie!='vacio') & (df.etiqueta=='vacio')].drop_duplicates().shape[0]
Total_Tr_Unic_CS_SA

#### Total de elementos en almacén Trazables Únicos sin NSerie, con Nactivo

In [ ]:
Total_Tr_Unic_SS_CA=df[['serie','etiqueta']].loc[(df.serie=='vacio') & (df.etiqueta!='vacio')].drop_duplicates().shape[0]
Total_Tr_Unic_SS_CA

### AGREGAR DEMÁS KPIS

In [ ]:
#Ajustar el df contra los kpis de la siguiente tabla:

KPIs=pd.DataFrame({'KPI':['Total Elementos','Total Elementos Trazables',
                         'Total NO Trazables','Total Trazables Unicos',
                         'Total Trazables Duplicados','Trazables Únicos en DHL Tepotzotlan',
                          'Total CS CA','Total CS SA','Total SS CA'],
                  'Resultado':[Total_Elementos,Total_Tr,Total_NOTr,
                              Total_Tr_Unic,Total_Tr_Dupli,
                               Total_Tr_Unic_CS_CA,Total_Tr_Unic_CS_SA,
                              Total_Tr_Unic_SS_CA]})

KPIs

In [ ]:
sc.stop()